## Harjoitustyö

## Vaihe 1

In [11]:
# tuodaan tarvittavat kirjastot

import requests
import json
import pprint
import pandas as pd
import re

#### Tehtävä 1. Hae Digitrafficin REST-rajapintapalvelusta (JSON-formaatissa) aktiiviset junat pääteosoitteella `/live-trains` Helsingin asemalta, joiden hakuparametrit ovat seuraavat:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [12]:
# määritetään rajapinnan osoite
train_url = 'https://rata.digitraffic.fi/api/v1/live-trains/station/'


# tallennetaan vastaus rajapinnasta muuttujaan
train_r = requests.get(url=train_url)


# hakuparametrien määrittely url-osoitteelle
station_shortcode = "HKI"
search_query_arrived = "arrived_trains=10"
search_query_departed = "departed_trains=10"
search_query_nonstopping = "include_nonstopping=False"


# määritetään hakuparametrien perusteella uusi url-osoite
new_url = "{0}{1}?{2}&{3}&{4}".format(train_url, station_shortcode, search_query_arrived, search_query_departed, search_query_nonstopping)


# tehdään uusi kysely hakuparametreilla muodostetusta url-osoitteesta ja tulostetaan rajapinnan vastauskoodi konsoliin
resp = requests.get(url=new_url)
print("Vastauskoodi:", resp.status_code)


Vastauskoodi: 200


#### Tehtävä 2. Tämän jälkeen lue kyseinen data Pandasin dataframeen ilman muokkaamista, jolloin sarakkeiden otsikot ovat seuraavanlaiset:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version` 

In [13]:
# luodaan Pandas-dataframe kyselyn tuloksesta
data = resp.text
df = pd.read_json(data)

#### Tehtävä 3. Poista kaikki muut sarakkeet paitsi `timeTableRows` ja `trainNumber`

In [14]:
# poistetaan kaikki muut sarakkeet paitsi 'timeTableRows' ja 'Trainnumber'
df2 = df.drop(columns=
              ['cancelled', 'commuterLineID', 'departureDate', 'operatorShortCode',
               'operatorUICCode', 'runningCurrently', 'timetableAcceptanceDate',
               'timetableType', 'trainCategory', 'trainType', 'version'])

#### Tehtävä 4. Tämän jälkeen pura `timeTableRows` sarakkeen arvot jokaisesta junasta erikseen (sarakkeen arvo jokaiselle junalle sisältää useampia JSON-objekteja) ja lisää dataan mukaan junan tiedot yksilöivä `trainNumber`

In [15]:
# avataan json data json_normalizen avulla
data_json_list = json.loads(data)
data_json = data_json_list
df5 = pd.json_normalize(data_json, record_path='timeTableRows', meta=['trainNumber'])

#### Tehtävä 5. Pudota sitten seuraavat sarakkeet datasta: `causes`, `estimateSource`, `liveEstimateTime`, `differenceInMinutes`, `commercialStop` ja `trainReady`

In [16]:
# pudotetaan halutut rivit pois
df6 = df5.drop(columns=['causes', 'estimateSource', 'liveEstimateTime','differenceInMinutes',
                        'commercialStop','trainReady.source', 'trainReady.accepted','trainReady.timestamp'])

#### Tehtävä 6. Täytä jokaisen rivin sarakkeen `actualTime` puuttuvat arvot `scheduledTime` -sarakkeen arvolla.

In [17]:
# täytetään NaN-arvot actualTime-sarakkeesta scheduledtime-sarakkeen arvoilla
df6['actualTime'].fillna(df6['scheduledTime'], inplace=True)

# muutetaan vielä koko dataframe object-muotoon
df7 = df6.astype('object')

## Vaihe 2

#### Tehtävä 7. Pura tämän kansion sisällä olevan `data.html` -tiedoston `table`-elementin solujen arvot käyttäen regex-syntaksia ja yhdistä ne vaiheessa 1 luotuun dataframeen. Huom: tehtävässä tulee käyttää vain regexiä (Esim. Pandasin read_html()-metodin käyttö on kielletty)!

In [25]:
# tuodaan tarvittavat kirjastot
import os
import urllib

f = urllib.request.urlopen("file:///C:/Users/ville/Projects/repos/ttvd0100-harjoitukset/harjoitustyo/harjoitustyo_data.html")
text = f.read()
text = text.decode()


# siivotaan tekstiversiota regexillä
stripped_text = re.sub("/\n|\s{2,}", ' ', text)
stripped_text2 = re.sub("[(\\n)]", '', stripped_text)
stripped_text3 = stripped_text2.replace(" ", "")


# etsitään tekstistä regexin avulla sarakkeet listaan
f_columns = re.findall(r"\<th>(.*?)\<\/th>", stripped_text3)
f_columns


# etsitään tekstistä sarakkeiden määrän mukaan sisältö sarakkeisiin ja tehdään siitä dataframe
f_content = re.findall(r"\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>\<td>(.*?)\<\/td>", stripped_text3)
df1 = pd.DataFrame(f_content)


# yhdistetään sarakkeet listasta luotuun DataFrameen
df1.columns = f_columns
df6.dtypes


# yhdistetään dataframe aiemmin luotuun
df_final = df6.append(df1)

df1

,actualTime,cancelled,commercialTrack,countryCode,scheduledTime,stationShortCode,stationUICCode,trainStopping,type,trainNumber
0,2019-08-06T04:48:00.000Z,False,,FI,2019-08-06T04:48:00.000Z,JVS,1272,False,DEPARTURE,150
1,2019-08-06T06:04:39.000Z,False,,FI,2019-08-06T06:04:39.000Z,R702,98998,False,ARRIVAL,150
2,2019-08-06T07:03:12.000Z,False,,FI,2019-08-06T07:03:12.000Z,ILA,98998,9,ARRIVAL,8337


## Vaihe 3

#### Tehtävä 8. Tee SQLite tietokanta ja lisää sinne vaiheessa 2 luotu dataframe.


In [ ]:
# tuodaan sqlite3 moduuli
import sqlite3 as sql

# luodaan tietokanta 'traindata', johon säilötään live-junien tiedot
conn = sql.connect('traindata.db')
df_final.to_sql('df_final', conn)

In [ ]:
# tarkistetaan, että data meni tietokantaan tekemällä haku
traindata = pd.read_sql('SELECT * FROM df_final', conn)
traindata